<a href="https://colab.research.google.com/github/wyattowalsh/sports-analytics/blob/main/basketball/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'> Basketball Data Collection </h1>

This notebook contains the associated work necessary to collect the data that composes the [***Kaggle Basketball Dataset*** (wyattowalsh/basketball)](https://www.kaggle.com/wyattowalsh/basketball) and serves as the foundation for the [basketball related projects](https://github.com/wyattowalsh/sports-analytics/tree/main/basketball) within my [sports analytics GitHub repository](https://github.com/wyattowalsh/sports-analytics).

One of the goals for the data collection component of this project is to produce a `robust`, *organized* dataset that can grow to as **large of a scale** as possible. You can find an explanation of my solution for storing the files related to the [***Basketball Dataset***](https://www.kaggle.com/wyattowalsh/basketball) below.

<img src="https://unsplash.com/photos/Kv-gAzpUSRg/download?force=true">

## Overview

***Kaggle*** offers many formats of which one can save files to a dataset, which include: `CSV`, `JSON`, `SQLite`, and `Archives`, among others. The platform essentially acts similarly to industrial cloud solutions like *Google Cloud Platform's* (**GCP**) ***Cloud Storage*** or *Amazon Web Service's* (**AWS**) ***S3*** albeit with a **100GB** storage capacity. ***Kaggle*** datasets as well as these industrial solutions can be considered as broad object/file storage and in certain data engineering paradigms can serve as data lakes. 

It seems that many state-of-the-art (SOTA) data storage solutions pivot around an organizational-wide data lake (of which itself allows for general object storage) that has multiple inputs (*"tributaries"*) both streaming into and routinely added to the overall lake. One benefit of this paradigm is that the lake facilitates the storage of both structured (tabular) and unstructured (image, video, audio, text, etc) data. This can prove useful because, as time progresses, new techniques for extracting useful information from unstructured data can be utilized. Thus it also seems like a good idea to hold onto all extracted data, if possible. 

***Kaggle*** datasets can serve as data lakes through the archival process or simply by storing data files in their raw file format. This certainly serves as a strong foundation for building a &#8212; one day in the future &#8212; <b><i>"big data"</i></b> collection. 

However, there is further work that can be done in configuring ***Kaggle*** datasets to enable additional platform functionality as well as improved storage efficiency. Structured data, whether structured upon extraction or structured through some pre-processing, can be stored in a ***SQLite*** database (`.sqlite` file type) as opposed to storing individual files such as `CSVs` or `JSONs` within the dataset. Thus, a single database file is stored as an object within the dataset, enabling additional functionality. One easily discerned advantage with storing in ***SQLite*** is that histograms of the distribution of across continuous variables are given directly within ***Kaggle***. 

As this project moves forward, I hope to collect a large collection of both structured and unstructured data. I hope that the ***SQLite*** database (`basketball.sqlite`) can serve to house the structured data in an efficient, useful format, similarly to the [***European Soccer Database***](https://www.kaggle.com/hugomathien/soccer).

## View System Information

In [1]:
print("********************** CUDA Version ********************** \n - \n")
!nvcc --version
print("********************** CPU Info ********************** \n - \n")
!cat /proc/cpuinfo
print("********************** CPU Count ********************** \n - \n")
import os
print(os.cpu_count())
print("********************** GPU Info ********************** \n - \n")
!nvidia-smi
print("********************** Python Version ********************** \n - \n")
!python -V

********************** CUDA Version ********************** 
 - 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
********************** CPU Info ********************** 
 - 

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_singl

## Prepare Development Environment

### Clone Project Repository and Install Dependencies

In [25]:
# remove sample data and clone repo
!rm -r sample_data/
!rm -r sports-analytics/
!git clone https://github.com/wyattowalsh/sports-analytics.git

# change directory to directory that contains this notebook
%cd /content/sports-analytics/basketball/notebooks/

# install dependencies
!pip install -r ../../dependencies/basketball/data_collection.txt
!pip install dask -U
!pip install distributed -U

rm: cannot remove 'sample_data/': No such file or directory
rm: cannot remove 'sports-analytics/': No such file or directory
Cloning into 'sports-analytics'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 321 (delta 116), reused 223 (delta 51), pack-reused 0
Receiving objects: 100% (321/321), 108.19 KiB | 2.35 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/content/sports-analytics/basketball/notebooks
     |████████████████████████████████| 931kB 3.6MB/s 
  Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0


     |████████████████████████████████| 675kB 5.8MB/s 
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


### Import Dependencies and Enable Tools

In [1]:
# nba_api dependencies
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats

# datascience stack
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sqlite3 as sql
import dask
from dask.distributed import Client, progress, LocalCluster
from dask.diagnostics import ProgressBar

# system utility stack
import os
import time
import urllib
from pyngrok import ngrok
from functools import partial

# register dask progress bar
pbar = ProgressBar()
pbar.register()

# Upload kaggle.json to /content/
from google.colab import files
uploaded = files.upload()

# Move and change permissions as needed, allowing for import
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
import kaggle

# change directory to directory that contains this notebook
%cd /content/sports-analytics/basketball/notebooks/

# utilize Colab Monitor
from urllib.request import urlopen
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()

mv: cannot stat 'kaggle.json': No such file or directory
/content/sports-analytics/basketball/notebooks
Now live at : http://colab-monitor.smankusors.com/60582f41004bb


## Collect Data

### Connect to Database

In [2]:
conn = sql.connect('../data/basketball.sqlite')

### Players

#### Get Players DataFrame and Type ID as String

In [5]:
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [6]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4501 non-null   object
 1   full_name   4501 non-null   object
 2   first_name  4501 non-null   object
 3   last_name   4501 non-null   object
 4   is_active   4501 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 145.2+ KB


#### Add Dataframe as Table to Database, Unless it Already Exists

In [8]:
try:
  df_players.to_sql('Player', conn)
except:
  pass

### Teams

#### Get Teams DataFrame, Type ID as String and Convert Year to Datetime

In [9]:
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams['year_founded'] =  pd.to_datetime(df_teams['year_founded'], format='%Y').dt.year # convert year to datetime type
df_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [10]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     object
 1   full_name     30 non-null     object
 2   abbreviation  30 non-null     object
 3   nickname      30 non-null     object
 4   city          30 non-null     object
 5   state         30 non-null     object
 6   year_founded  30 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


#### Add Dataframe as Table to Database, Unless it Already Exists

In [11]:
try:
  df_teams.to_sql('Team', conn)
except:
  pass

### Common Player Information

#### Get Proxy Servers

In [34]:
cluster = LocalCluster(n_workers=64)
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:42145 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 27.39 GB


In [35]:
from google.colab import output
output.serve_kernel_port_as_window(8787)

<IPython.core.display.Javascript object>

In [31]:
!wget -O http_proxies.txt "https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=yes&anonymity=all&simplified=true"

with open('http_proxies.txt', 'r') as file:
  proxies = file.read().split('\n')
print("Original number of proxies: ", len(proxies))

def check_proxies(proxy):
  try:
    urllib.request.urlopen("http://" + proxy, timeout = 30)
    print("alive proxy detected")
    return proxy
  except:
    pass

tested_proxies = []
for proxy in proxies:
  tested_proxy = dask.delayed(check_proxies)(proxy)
  tested_proxies.append(tested_proxy)
proxies = dask.persist(*tested_proxies)
with ProgressBar():
  proxies = dask.compute(proxies)
c.shutdown()
c.close()
proxies = [proxy for proxy in proxies if proxy != None]
print("Number of proxies alive: ", len(proxies))

--2021-03-22 06:13:26--  https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=yes&anonymity=all&simplified=true
Resolving api.proxyscrape.com (api.proxyscrape.com)... 151.139.128.11
Connecting to api.proxyscrape.com (api.proxyscrape.com)|151.139.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2295 (2.2K) [text/plain]
Saving to: ‘http_proxies.txt’

http_proxies.txt    100%[===================>]   2.24K  --.-KB/s    in 0s      

2021-03-22 06:13:26 (27.9 MB/s) - ‘http_proxies.txt’ saved [2295/2295]

Original number of proxies:  112
Number of proxies alive:  1


#### Extract Data

In [ ]:
def get_common_player_info(player_id, proxies):
  no_res = True
  proxy_collection_counter = 0
  proxy_index = 0
  while no_res:
    try:
      res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=10)
      no_res = False
    except:
      while no_res:
        try: 
          res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, proxy="http://" + proxies[proxy_index], timeout=15)
          no_res = False
        except:
          if (proxy_index + 1) >= len(proxies):
            proxy_index = 0
            proxy_collection_counter = proxy_collection_counter + 1
            print("proxies failed for {}, {} times; Getting more proxies...".format(player_id, proxy_collection_counter))
            break
          else:
            proxy_index = proxy_index + 1

dfs = []
player_ids = pd.read_sql('SELECT id FROM Player', conn).T.values[0]
for player_id in player_ids[0:4]:
  dfs.append(dask.delayed(partial(get_common_player_info, proxies=proxies))(player_id))
dfs = dask.persist(*dfs)
with ProgressBar():
  dfs = dask.compute(dfs)
c.shutdown()
c.close()
dfs = [df for df in dfs if df != None]
df = pd.concat(dfs)
df.head()

In [37]:
player_ids = pd.read_sql('SELECT id FROM Player', conn).T.values[0]
for player_id in player_ids[0:4]:
  print(player_id)

76001
76002
76003
51


In [33]:
c.shutdown()
c.close

<bound method Client.close of <Client: not connected>>

### Define Function to Scrape New Proxy List and Return Proxies Tested to be Alive

In [ ]:
def get_proxies():
    !wget -O http_proxies.txt "https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=5000&country=all&ssl=yes&anonymity=all&simplified=true"

    with open('http_proxies.txt', 'r') as file:
        proxies = file.read().split('\n')
    print("Original number of proxies: ", len(proxies))

    def check_proxies(proxy):
        try:
            urllib.request.urlopen("http://" + proxy, timeout = 30)
            print("alive proxy detected")
        except:
            return proxy

    dead_proxies = []
    for proxy in proxies:
        dead_proxy = dask.delayed(check_proxies)(proxy)
        dead_proxies.append(dead_proxy)

    dead_proxies = dask.persist(*dead_proxies)
    dead_proxies = list(filter(None, dask.compute(dead_proxies))) 

    [proxies.remove(proxy) for proxy in dead_proxies if proxy in proxies]
    if "" in proxies:
        proxies.remove("")
    print("Number of proxies alive: ", len(proxies))
    return proxies

### Create Dask Cluster with the Number of Workers Equal to the Number of CPU Cores

In [ ]:
# Make sure to put appropiate number of workers given info provided in the output of the first cell
cluster = LocalCluster(n_workers=32) 
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:41867 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 32 Memory: 27.39 GB


## Process `get_proxies()` with Dask then Shutdown Cluster

In [ ]:
proxies = get_proxies()
with open('valid_proxies.txt', 'w') as f:
    for proxy in proxies:
        f.write("%s\n" % proxy)
c.shutdown()
proxies

--2021-03-19 17:02:50--  https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=5000&country=all&ssl=yes&anonymity=all&simplified=true
Resolving api.proxyscrape.com (api.proxyscrape.com)... 151.139.128.11
Connecting to api.proxyscrape.com (api.proxyscrape.com)|151.139.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1527 (1.5K) [text/plain]
Saving to: ‘http_proxies.txt’

http_proxies.txt    100%[===================>]   1.49K  --.-KB/s    in 0s      

2021-03-19 17:02:51 (22.1 MB/s) - ‘http_proxies.txt’ saved [1527/1527]

Original number of proxies:  76
Number of proxies alive:  75


['91.149.203.9:3128',
 '47.254.247.210:3128',
 '51.79.173.166:3128',
 '51.79.145.108:8080',
 '51.158.172.165:8811',
 '51.210.150.171:3128',
 '144.91.89.199:3128',
 '62.210.203.211:8080',
 '206.189.153.241:3128',
 '95.217.102.133:3128',
 '134.209.130.135:10303',
 '103.125.162.1:83',
 '167.172.109.12:36499',
 '142.44.243.113:3128',
 '104.248.146.99:3128',
 '65.1.148.79:80',
 '134.209.102.189:8888',
 '111.235.65.211:45811',
 '116.58.236.172:8080',
 '88.220.104.178:8080',
 '186.96.170.132:9991',
 '51.159.24.172:3162',
 '178.32.247.156:3128',
 '176.241.129.113:3128',
 '81.90.224.248:3128',
 '51.158.172.165:8761',
 '190.83.114.9:999',
 '103.52.145.97:8080',
 '91.221.74.150:3128',
 '122.155.165.191:3128',
 '191.252.61.219:3128',
 '82.200.181.54:3129',
 '187.1.174.94:20183',
 '18.139.161.101:80',
 '51.79.173.154:8080',
 '51.79.144.52:8080',
 '103.152.101.136:8080',
 '51.79.173.167:8080',
 '20.195.17.90:3128',
 '13.212.154.38:80',
 '206.189.153.241:8888',
 '176.113.73.101:3128',
 '83.168.86.189

## Get Common Player Information

### Define Functions `get_quick_proxies()` & `get_common_player_info()`

Each function utilizes a ***Dask*** cluster. 

`get_quick_proxies()` gets a list of proxies (tested to be alive) more quickly than the function above. This function is used in the case that all proxies found from the above function fail to return responses from stats.nba.com. 

`get_common_player_info()` returns dataframe of common player infomation for a certain player. The paradigm here is to distribute jobs (where each job is collecting common player info for a certain player) across a ***Dask*** cluster since all outputs will be the same and can be easily be concatenated. 

In [ ]:
def get_common_player_info(player_id):
  with open('valid_proxies.txt', 'r') as file:
    proxies = file.read().split('\n')
  res_dfs = []
  i = 0
  proxies_retrieved = False
  # while response is empty
  while len(res_dfs) <= 0: 
    # try the request without a proxy
    try:
      res_dfs = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=100).get_data_frames()
      res_df = pd.merge(res_dfs[0], res_dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
      res_df = res_df.drop(['TimeFrame'], axis=1)
      print("******* SUCCESS ******* \n ******* {} ******* \n".format(player_id))
      return res_df
    # if still fails, then try with proxy
    except:
      try:
        res_dfs = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout=150).get_data_frames()
        res_df = pd.merge(res_dfs[0], res_dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
        res_df = res_df.drop(['TimeFrame'], axis=1)
        print("******* SUCCESS ******* \n ******* {} ******* \n".format(player_id))
        return res_df
      # if still fails, move on to next proxy, unless out of proxies
      except:
        if (i + 1) < len(proxies):
          i = i + 1
        # if out of proxies, restart counter and get new proxies
        else:
          if proxies_retrieved:
            print("******* FAILURE ****** \n ****** {} ******* \n ******* RETURNING NONE ******".format(player_id))
            return None
          else:
            print("******* FAILURE ****** \n ****** {} ******* \n ******* COLLECTING NEW PROXIES AND TRYING REQUEST AGAIN ******".format(player_id))
            i = 0
            proxies = get_proxies()
            proxies_retrieved = True
      

### Extract Common Player Information for all Players

In [ ]:
# Make sure to put appropiate number of workers given info provided in the output of the first cell
def main():
    cluster = LocalCluster(n_workers=12) 
    with Client(address=cluster):
      conn = sql.connect('../data/basketball.sqlite')
      player_ids = pd.read_sql('SELECT id FROM Player', conn).values #pd.DataFrame(players.get_players()).astype({'id': 'str'})['id'].values

      dfs = []
      for player_id in player_ids:
        df = dask.delayed(get_common_player_info(player_id))
        dfs.append(df)

      dfs = dask.persist(*dfs)
      dfs = dask.compute(dfs)
      dfs = [df for df in dfs if df != None]
      dfs = dask.dataframe.multi.concat(dfs)
      return dfs

if __name__ == "__main__":
  common_player_info_dfs = main()

common_player_info_dfs.head()

In [ ]:
cluster.shutdown()